<a href="https://colab.research.google.com/github/allnes/age_classifier/blob/master/train_age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [0]:
from google.colab import drive
import os, natsort as nsrt, numpy as np
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt

drive.mount('/content/drive')
PATH_PROJECT='/content/drive/My Drive/DL_DATA_GRAPH/'
PATH_CNN_REPO=PATH_PROJECT + 'BUILD/cnn_graph/'
os.chdir(PATH_CNN_REPO)
from lib import models, graph, coarsening, utils
%ls

# !git clone https://github.com/mdeff/cnn_graph
!git pull origin master

os.chdir(PATH_PROJECT)
%ls
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
lib/         makefile   rcv1.ipynb  requirements.txt  usage.ipynb
LICENSE.txt  nips2016/  README.md   trials/
From https://github.com/mdeff/cnn_graph
 * branch            master     -> FETCH_HEAD
Already up to date.
BUILD/  DATA/


## Preprocessing data

In [0]:
PATH_GRAPHS=PATH_PROJECT + 'DATA/mini_graphs/graphs/'
PATH_LABELS=PATH_PROJECT + 'DATA/mini_graphs/GSE87571_samples.txt'

list_grpahs = []
for (_, _, filenames) in os.walk(PATH_GRAPHS):
    list_grpahs = list_grpahs + filenames

list_grpahs = nsrt.natsorted(list_grpahs)[0::2]

raw_graph = np.load(PATH_GRAPHS + list_grpahs[0])
raw_edges = raw_graph['E'].transpose()
rows = raw_edges[0]
cols = raw_edges[1]
raw_data = raw_graph['D']
sparse_graph = coo_matrix((raw_data, (rows, cols)),
                          shape=(raw_graph['num_features'],
                                 raw_graph['num_features']))

print(raw_graph.files)
print(sparse_graph.shape)


dist, idx = graph.distance_sklearn_metrics(X_train.T, k=10, metric='euclidean')
A = graph.adjacency(dist, idx).astype(np.float32)
# for graph_name in list_grpahs:
#     raw_graph = np.load(PATH_GRAPHS + graph_name)

['M', 'D', 'E', 'num_samples', 'num_features']
(14768, 14768)


NameError: ignored